In [ ]:
%run 'Data Cleaning.ipynb'

In [ ]:
import torch
import torch.nn as nn

class CollabFiltModel(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
    
    def forward(self, user, item):
        user_emb = self.user_emb(user)
        item_emb = self.item_emb(item)
        return (user_emb * item_emb).sum(1)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

cleaned_df['User_id'] = user_encoder.fit_transform(cleaned_df['User_id'])
cleaned_df['Title'] = item_encoder.fit_transform(cleaned_df['Title'])

class ReviewDataset(Dataset):
    def __init__(self, users, items, ratings):
        self.users = torch.tensor(users, dtype=torch.int64)
        self.items = torch.tensor(items, dtype=torch.int64)
        self.ratings = torch.tensor(ratings, dtype=torch.float32)
    
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]
    
    def __len__(self):
        return len(self.ratings)

dataset = ReviewDataset(cleaned_df['User_id'].values,
                        cleaned_df['Title'].values,
                        cleaned_df['review/score'].values)

dataloader = DataLoader(dataset, batch_size=512, shuffle=True)


In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = CollabFiltModel(num_users=cleaned_df['User_id'].nunique(),
                        num_items=cleaned_df['Title'].nunique()).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epoch_loss_dict = {}
for epoch in range(10):
    for users, items, ratings in dataloader:
        users, items, ratings = users.to(device), items.to(device), ratings.to(device)
        
        optimizer.zero_grad()
        predictions = model(users, items)
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    epoch_loss_dict[epoch] = loss.item()

In [ ]:
def recommend_items_for_user(model, user_id, user_encoder, item_encoder, num_recommendations=5):
    # Set model to evaluation mode
    model.eval()
    
    # Encode the user ID
    encoded_user_id = user_encoder.transform([user_id])
    
    # Create a tensor for the user ID
    user_tensor = torch.tensor(encoded_user_id, dtype=torch.int64).to(device)
    
    # Predict scores for all items for this user
    all_items = torch.tensor(range(len(item_encoder.classes_)), dtype=torch.int64).to(device)
    user = user_tensor.repeat(len(all_items))
    predictions = model(user, all_items).detach().cpu().numpy()
    # Get the top N recommendations; argsort returns indices of sorted array
    top_indices = predictions.argsort()[-num_recommendations:][::-1]
    # Decode the recommended item IDs
    recommended_items = item_encoder.inverse_transform(top_indices)
    top_ratings = predictions[top_indices]
    print(top_ratings)
    return recommended_items

user_id_to_recommend = 'AVCGYZL8FQQTD'  # Example user ID
recommended_items = recommend_items_for_user(model, user_id_to_recommend, user_encoder, item_encoder)
print("Recommended Items:", recommended_items)


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(cleaned_df, test_size=0.2, random_state=42)


In [ ]:
train_dataset = ReviewDataset(train_df['User_id'].values, train_df['Title'].values, train_df['review/score'].values)
train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True)

test_dataset = ReviewDataset(test_df['User_id'].values, test_df['Title'].values, test_df['review/score'].values)
test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=False)


In [ ]:
model.eval()
total_squared_error = 0.0
total_absolute_error = 0.0
total_count = 0

with torch.no_grad():
    for users, items, ratings in test_dataloader:
        users, items, ratings = users.to(device), items.to(device), ratings.to(device)
        predictions = model(users, items)
        
        # Squared error for RMSE
        squared_error = (predictions - ratings) ** 2
        total_squared_error += squared_error.sum().item()
        # Absolute error for MAE
        absolute_error = torch.abs(predictions - ratings)
        total_absolute_error += absolute_error.sum().item()
        
        total_count += ratings.size(0)

# Calculate mean squared error and RMSE
mse = total_squared_error / total_count
rmse = torch.sqrt(torch.tensor(mse))

# Calculate mean absolute error
mae = total_absolute_error / total_count

print(f'RMSE: {rmse}')
print(f'MAE: {mae}')


In [ ]:
# Save the model state dictionary
torch.save(model.state_dict(), 'collab_filt_model_state_dict.pth')

# Save the encoders
import joblib
joblib.dump(user_encoder, 'user_encoder.joblib')
joblib.dump(item_encoder, 'item_encoder.joblib')

cleaned_df.to_csv('cleaned_df')


In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from copy import deepcopy
import os

In [ ]:
import scipy.sparse as sp

def save_sparse_matrix(tensor, path):
    # Convert PyTorch tensor to a NumPy array
    tensor_np = tensor.detach().cpu().numpy()
    
    # Convert the NumPy array to a SciPy sparse matrix (CSR format)
    sparse_matrix = sp.csr_matrix(tensor_np)
    
    # Save the sparse matrix to disk
    sp.save_npz(path, sparse_matrix)

# Assuming pruned_model is your pruned model
save_sparse_matrix(model.user_emb.weight, "user_emb_sparse.npz")
save_sparse_matrix(model.item_emb.weight, "item_emb_sparse.npz")